In [1]:
%pwd

'c:\\Users\\BOSS\\Desktop\\Chatbot project\\End-to-End-Chatbot-project\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\BOSS\\Desktop\\Chatbot project\\End-to-End-Chatbot-project'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
## Load pdf files
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob = "*.pdf",
        loader_cls=PyPDFLoader
    )
    return loader.load()

In [7]:
docs = load_pdf_file(data = "data")

In [35]:
# Text split
def text_split(extracted_data):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size = 330,
        chunk_overlap = 20
    )
    chunk = splitter.split_documents(extracted_data)
    return chunk

In [36]:
text_chunks = text_split(docs)
print(len(text_chunks))

512


In [38]:
text_chunks[0].page_content

'25/04/2022 The Constitution of the People   ’s Republic of Bangladesh\nPART I\nTHE REPUBLIC\nThe Constitution of the People   ’s Republic of Bangladesh\n( ACT NO. OF 1972 )\n[BISMILLAH-AR-RAHMAN-AR-RAHIM1\n (In the name of Allah, the Beneficient, the Merciful)/\n In the name of the Creator, the Merciful.]\n  PREAMBLE'

In [42]:
from langchain.embeddings import HuggingFaceEmbeddings

In [49]:
embedding = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)

In [51]:
query_result = embedding.embed_query("Hello How are you")
print(query_result)
print(len(query_result))

[-0.03483875095844269, 0.060805998742580414, 0.09206905215978622, 0.04381444677710533, -0.04384097084403038, -0.08060231804847717, 0.05162295326590538, 0.00019532481383066624, -0.09116464108228683, 0.03794107213616371, -0.017033925279974937, 0.0010522498050704598, -0.0174382496625185, -0.03719784691929817, 0.037271659821271896, -0.02864157408475876, 0.1004432663321495, -0.08060581982135773, -0.12832653522491455, 0.03520473465323448, -0.04183833301067352, 0.02712908573448658, 0.004000987391918898, 0.048280805349349976, -0.05778234824538231, -0.04523955658078194, 0.011549552902579308, 0.009121071547269821, 0.05104982852935791, -0.0879344493150711, -0.024926627054810524, 0.038468506187200546, -0.004912429489195347, 0.014689882285892963, -0.0196964330971241, 0.02064572088420391, -0.011695235967636108, -0.12021058052778244, -0.0024256850592792034, -0.015537234954535961, 0.0075797438621521, -0.07284560799598694, -0.005615473259240389, -0.03771218657493591, 0.07821177691221237, -0.06151863560

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

In [54]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [55]:
pc = pinecone(
    api_key=PINECONE_API_KEY
)

In [57]:
index_name = "constitution"


pc.create_index(
    name=index_name,
    dimension = 384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "constitution",
    "metric": "cosine",
    "host": "constitution-9yj22ay.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [58]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [60]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    embedding=embedding,
    index_name = index_name,
    documents = text_chunks
)

In [61]:
# load index

vector_store_load = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [73]:
retriever = vector_store_load.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 5}
)

In [74]:
retriever.invoke("What is the state language?")

[Document(id='aa1a6f44-b916-45f8-b6d1-a5ebfc79ab9c', metadata={'creationdate': '2022-04-25T06:57:54+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36', 'moddate': '2022-04-25T06:57:54+00:00', 'page': 1.0, 'page_label': '2', 'producer': 'Skia/PDF m100', 'source': 'data\\Constitution_Bangladesh.pdf', 'total_pages': 72.0}, page_content='The state\nreligion\n[2A. The state religion of the Republic is Islam, but the State shall ensure\nequal status and equal right in the practice of the Hindu, Buddhist,\nChristian and other religions.]\n5\nThe state\nlanguage\n3. The state language of the Republic is [Bangla].6\nNational\nanthem,\nflag and\nemblem'),
 Document(id='9a73d947-d23e-44b0-8bca-14f4e08d719b', metadata={'creationdate': '2022-04-25T06:57:54+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36', 'moddate': '2022-04-

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [79]:
llm_endpoint = HuggingFaceEndpoint(
    repo_id= "meta-llama/Llama-3.3-70B-Instruct"
)
model = ChatHuggingFace(llm = llm_endpoint)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate

In [81]:
system_prompt = (
    "You are an AI assistant for question-answering tasks"
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"    
)

In [133]:
prompt = ChatPromptTemplate([
    ('system' , system_prompt),
    ('placeholder', "{chat_history}"),
    ('human', "{input}")
])

In [134]:
qa_chain = create_stuff_documents_chain(llm = model, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
chat_history = memory.load_memory_variables({})["chat_history"]

question = "who iam i?"

input = {
    "input" : question,
    "chat_history" : chat_history
}

response = rag_chain.invoke(input)
answer = response["answer"]
memory.save_context({"input" : question},{"answer" : answer})

print(answer)

You are Mahadi Hasan.


I don't know your name. The provided context only mentions the name of a country, "Bangladesh", and does not contain any information about you. There is no personal data available to determine your name.
